In [1]:
import importlib
import ifimage_tools
importlib.reload(ifimage_tools)
import pandas as pd
import warnings



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.4 
platform:       	linux 
python version: 	3.10.16 
torch version:  	2.7.0+cu126! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




In [2]:
import os
import numpy as np
from stardist.matching import matching
image_dir = "Reorgnized Ground Truth"
masks_dir = "Reorgnized Ground Truth/mask"
dataset = ifimage_tools.IfImageDataset(image_dir, masks_dir, {})
dataset.load_data()

old_version_sample_ids = ["6390", "8408", "8406", "8405v2", "8405", "8407"]
for sample_id in old_version_sample_ids:
    if sample_id in dataset.samples:
        del dataset.samples[sample_id]

In [3]:
from preprocessing import SamplePreprocessor,batch_process_zip_rois,batch_rename

In [4]:
import os
import shutil

def collect_ground_truth(src_root, dest_dir, exts=(".npy", ".tiff", ".tif")):
    """
    Recursively copy all files ending with exts from src_root into dest_dir.
    """
    os.makedirs(dest_dir, exist_ok=True)
    for root, _, files in os.walk(src_root):
        for f in files:
            if f.lower().endswith(exts):
                src_path = os.path.join(root, f)
                dst_path = os.path.join(dest_dir, f)
                # 如果你担心重名被覆盖，可以改成：
                # rel = os.path.relpath(root, src_root).replace(os.sep, "_")
                # dst_path = os.path.join(dest_dir, f"{rel}_{f}")
                shutil.copy2(src_path, dst_path)
                print(f"📂 Copied {src_path} → {dst_path}")

if __name__ == "__main__":
    src = "/Users/macbookair/Downloads/new524/renamed"
    dst = "/Users/macbookair/PycharmProjects/ifimage/Reorgnized Ground Truth"
    collect_ground_truth(src, dst)
    print("🎉 All done, babe!")


📂 Copied /Users/macbookair/Downloads/new524/renamed/neun_1111/neun_1111_cellbodies.npy → /Users/macbookair/PycharmProjects/ifimage/Reorgnized Ground Truth/neun_1111_cellbodies.npy
📂 Copied /Users/macbookair/Downloads/new524/renamed/neun_1116/neun_1116_cellbodies.npy → /Users/macbookair/PycharmProjects/ifimage/Reorgnized Ground Truth/neun_1116_cellbodies.npy
📂 Copied /Users/macbookair/Downloads/new524/renamed/neun_1110/neun_1110_cellbodies.npy → /Users/macbookair/PycharmProjects/ifimage/Reorgnized Ground Truth/neun_1110_cellbodies.npy
📂 Copied /Users/macbookair/Downloads/new524/renamed/neun_1109/neun_1109_cellbodies.npy → /Users/macbookair/PycharmProjects/ifimage/Reorgnized Ground Truth/neun_1109_cellbodies.npy
📂 Copied /Users/macbookair/Downloads/new524/renamed/neun_1108/neun_1108_cellbodies.npy → /Users/macbookair/PycharmProjects/ifimage/Reorgnized Ground Truth/neun_1108_cellbodies.npy
📂 Copied /Users/macbookair/Downloads/new524/renamed/neun_1115/neun_1115_cellbodies.npy → /Users/macb

In [12]:
import zipfile
import os

# 1) List your sample IDs
sample_ids = ['1116','1111','1110','1108','1115','1112','1113','1114','1109']
zip_path = '/Users/macbookair/Downloads/NeuN.zip'
out_dir = 'extracted_tiffs'

os.makedirs(out_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zf:
    # Build a lookup for each sample
    results = {sid: {'dapi': None, 'marker': None} for sid in sample_ids}

    # 2) Scan all TIFFs once
    for info in zf.infolist():
        if info.is_dir() or not info.filename.lower().endswith('.tiff'):
            continue
        fname = os.path.basename(info.filename).lower()

        # Check each sample ID
        for sid in sample_ids:
            if sid in info.filename:
                if 'b0c0' in fname and results[sid]['dapi'] is None:
                    results[sid]['dapi'] = info
                elif 'b0c1' in fname and results[sid]['marker'] is None:
                    results[sid]['marker'] = info

    # 3) Extract the streams we found
    for sid, files in results.items():
        dapi_info   = files['dapi']
        marker_info = files['marker']

        if not dapi_info or not marker_info:
            print(f"[!] Sample {sid}:",
                  "DAPI missing"   if not dapi_info   else "",
                  "Marker missing" if not marker_info else "")
            continue

        # Write out
        with zf.open(dapi_info) as ds, open(os.path.join(out_dir, f"neun_{sid}.tiff"), 'wb') as out:
            out.write(ds.read())
        with zf.open(marker_info) as ms, open(os.path.join(out_dir, f"neun_{sid}_marker.tiff"), 'wb') as out:
            out.write(ms.read())

        print(f"✓ Extracted neun_{sid}.tiff and neun_{sid}_marker.tiff")


✓ Extracted neun_1116.tiff and neun_1116_marker.tiff
✓ Extracted neun_1111.tiff and neun_1111_marker.tiff
✓ Extracted neun_1110.tiff and neun_1110_marker.tiff
✓ Extracted neun_1108.tiff and neun_1108_marker.tiff
✓ Extracted neun_1115.tiff and neun_1115_marker.tiff
✓ Extracted neun_1112.tiff and neun_1112_marker.tiff
✓ Extracted neun_1113.tiff and neun_1113_marker.tiff
✓ Extracted neun_1114.tiff and neun_1114_marker.tiff
✓ Extracted neun_1109.tiff and neun_1109_marker.tiff


In [4]:
to_process = []
for sample_id, sample in dataset.samples.items():
    has_cellbodies = (sample.cellbodies_mask is not None) or (sample.cellbodies_multimask is not None)
    has_dapi_multi = sample.dapi_multi_mask is not None
    if has_cellbodies and not has_dapi_multi:
        to_process.append(sample_id)

print(f"—— 一共发现 {len(to_process)} 个样本，需要先跑 apply_nuc_pipeline，然后生成 ROI：{to_process}")


—— 一共发现 60 个样本，需要先跑 apply_nuc_pipeline，然后生成 ROI：['8550', '8917', '5191', '6833', '10166', '9106', '4642', '5792', '10061', '1110', '7925', '5863', '1112', '7739', '5059', '7113', '3569', '7685', '8224', '4515', '7870', '6790', '7962', '1111', '8746', '3532', '3999', '3527', '7071', '5923', '5789', '5794', '9755', '6020', '9170', '10015', '1116', '4238', '1114', '5795', '4548', '8517', '9783', '1108', '8310', '9472', '4736', '7319', '9866', '6212', '8942', '1113', '6748', '1115', '5410', '6523', '6466', '4683', '4319', '1109']


In [ ]:
# 输出目录
output_zip_dir = "roi_zips"
os.makedirs(output_zip_dir, exist_ok=True)

for sid in to_process:
    sample = dataset.samples[sid]
    print(f"\n[INFO] 开始处理样本：{sid}")

    # 1. 调用 apply_nuc_pipeline 生成 dapi_multi_mask
    print(f"  • 运行 apply_nuc_pipeline...")
    sample.apply_nuc_pipeline()

    if sample.masks is None:
        print(f"  [WARN] 样本 {sid} 在运行 apply_nuc_pipeline 后仍然没有 dapi_multi_mask，跳过。")
        continue

    # 2. 确保 dapi_multi_mask 是一个多标签 mask
    dapi_mask = sample.masks[""].copy()
    # 如果是 0/1 二值图，就先做 connected-components 标号
    from skimage.measure import label as sklabel
    if dapi_mask.max() <= 1:
        dapi_mask = sklabel(dapi_mask)

    # 3. 调用 mask_to_rois_zip，把 dapi_mask 转成 ImageJ ROI 并打包
    zip_path = os.path.join(output_zip_dir, f"{sid}_nuclei_roi.zip")
    print(f"  • 生成 ROI ZIP: {zip_path} ...")
    mask_to_rois_zip(dapi_mask, zip_path)

    print(f"  [OK] 样本 {sid} 的 nuclei ROI ZIP 已保存。")

print("\n全部样本处理完毕，请到 “roi_zips” 目录查看各个 *_nuclei_roi.zip 文件。")

model_type argument is not used in v4.0.1+. Ignoring this argument...



[INFO] 开始处理样本：8550
  • 运行 apply_nuc_pipeline...


model_type argument is not used in v4.0.1+. Ignoring this argument...
model_type argument is not used in v4.0.1+. Ignoring this argument...
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
model_type argument is not used in v4.0.1+. Ignoring this argument...


  [WARN] 样本 8550 在运行 apply_nuc_pipeline 后仍然没有 dapi_multi_mask，跳过。

[INFO] 开始处理样本：8917
  • 运行 apply_nuc_pipeline...


model_type argument is not used in v4.0.1+. Ignoring this argument...
model_type argument is not used in v4.0.1+. Ignoring this argument...
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used


KeyboardInterrupt: 